# University of Michigan 
# Master of Applied Data Science
### SIAD 699: Capstone
Team: James Yoon (jamyoon), Mario Feliciano (felicma), and James Tuccori (jtuccori)

Date: April 2023
____
# Social Media Sentiment & Predicting Trading Signals 

##### In this notebook, we take our top performing model (Stacking_LR_model_JT2019to2022TEST_news_google), and then use it to create buy and sell signals(prediction) for us to invest on.

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, date, timedelta
import random
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import mplfinance as mpf
import numpy as np

from datetime import datetime
import time

import tweepy
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import os
import glob

from alpaca_trade_api import REST, Stream
from transformers import pipeline
import pandas as pd
from textblob import TextBlob

import pandas as pd
import yfinance as yf
import numpy as np
import pytz
from dateutil import parser

from pytrends.request import TrendReq
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from adspy_shared_utilities import plot_decision_tree
from adspy_shared_utilities import plot_class_regions_for_classifier
from adspy_shared_utilities import plot_decision_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from adspy_shared_utilities import plot_decision_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
import pickle
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
from finta import TA
import shap
from keras.models import load_model

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.callbacks import EarlyStopping
import schedule
import ccxt
import pandas as pd
import numpy as np
import schedule
import time
import datetime
pd.set_option('display.max_rows', None)

import ta
from ta.volatility import AverageTrueRange
from ta.trend import sma_indicator

## 1. Hourly sampling extraction

#### To make our hourly trading both successful we are implementing an "Imcremental Data Update/Extract" strategy. Our overall hourly bot needs about 57 hours of data to be able to successfully predict because our longest moving average is calculated at 57 hour moving average in STEP 2 of our pipeline.  If we extract the last 57 hours each time we need to make a prediction , it wont be possible because it takes about 2 hours and 24 minutes to extract these twitter information.  So this block of code below extracts the last 60 hours of data only once and store that information as  CSV file, which we then call upon this file and add only the last hour of twitter extracts in hourly increments.

In [3]:
%%time

from tqdm import tqdm

##WORKING SO FAR V4
PasthoursLongterm = 60
start_time = datetime.now() - timedelta(hours=PasthoursLongterm)
day1 = start_time.timestamp()

number_of_tweets = 480
random_sample = 0.8
tweets_list2 = []

print("Scraping tweets for the last 60 hours...")
for j in tqdm(range(PasthoursLongterm), desc="Hours"):
    print (j)
    x = 0
    for tweet in sntwitter.TwitterSearchScraper(
        'bitcoin since_time:{} until_time:{}'.format(
            int(day1),
            int((start_time + timedelta(hours=j+1)).timestamp())
        )
    ).get_items():
        if x == number_of_tweets:
            break
        if random.random() > random_sample:
            tweets_list2.append(
                [
                    tweet.user.username,
                    tweet.rawContent,
                    tweet.date,
                    tweet.likeCount,
                    tweet.retweetCount,
                    tweet.user.followersCount,
                    tweet.user.friendsCount,
                ]
            )
            x += 1

tweets_df_last100hours = pd.DataFrame(tweets_list2, columns=['user', 'tweets', 'time', 'favorites', 'retweets', 'number_of_followers', 'following'])
print("Finished scraping tweets")

tweets_df_last100hours.to_csv('assets/runs/Last100hours.csv')

Scraping tweets for the last 60 hours...


Hours:   0%|                                                                                    | 0/60 [00:00<?, ?it/s]

0


Hours:   2%|█▏                                                                        | 1/60 [01:22<1:20:40, 82.03s/it]

1


...

58


Hours:  98%|██████████████████████████████████████████████████████████████████████▊ | 59/60 [2:21:49<02:29, 149.42s/it]

59


Hours: 100%|████████████████████████████████████████████████████████████████████████| 60/60 [2:24:21<00:00, 144.35s/it]


Finished scraping tweets
Wall time: 2h 24min 21s


In [15]:
tweets_df_last100hours.tail()

,user,tweets,time,favorites,retweets,number_of_followers,following
30475,MarketJane,https://t.co/ck6QxiG0Ol\n\n#Bitcoin is the bes...,2023-04-04 13:56:40+00:00,0,0,5177,502
30476,Bitcoin24ORE,BREAKING: una legge anti-Bitcoin (n 1751) è ap...,2023-04-04 13:56:28+00:00,1,0,250,81
30477,BullSkyRocket,🚨🚨 BREAKING 🚨🚨\nRalph Lauren To Accept #Bitc...,2023-04-04 13:56:21+00:00,0,0,163,47
30478,l_Waqas_l,@OTC_Bitcoin Let me know when you're not sleep...,2023-04-04 13:56:10+00:00,2,0,152,528
30479,DudeJLebowski,@CHAIRFORCE_BTC @ODELL Sounds like shitcoiners...,2023-04-04 13:56:09+00:00,4,0,10665,2756


## 2. Hourly Trading Bot

#### This code block below runs through the whole pipelines in hourly increments and provides us a prediction in the for either a Buy , Sell or hold signal. 

#### Starting in code line 865, the code block could be uncommented to link it to a binance account, to allow real time buy/sell training

In [9]:
%%time


def run_program():
    Pasthours = 1
    ##WORKING SO FAR V4
    start_time = datetime.now() - timedelta(hours=Pasthours)
    day1 = start_time.timestamp()

    number_of_tweets = 480
    random_sample = 0.8
    tweets_list2 = []

    print("Scraping tweets for the last 1 hour...")
    for j in range(Pasthours):
        print (j)
        x = 0
        for tweet in sntwitter.TwitterSearchScraper(
            'bitcoin since_time:{} until_time:{}'.format(
                int(day1),
                int((start_time + timedelta(hours=j+1)).timestamp())  # Add 1 hour to the start_time
            )
        ).get_items():
            if x == number_of_tweets:
                break
            if random.random() > random_sample:
                tweets_list2.append(
                    [
                        tweet.user.username,
                        tweet.rawContent,
                        tweet.date,
                        tweet.likeCount,
                        tweet.retweetCount,
                        tweet.user.followersCount,
                        tweet.user.friendsCount,
                    ]
                )
                x += 1

    tweets_df_last1hours = pd.DataFrame(tweets_list2, columns=['user', 'tweets', 'time', 'favorites', 'retweets', 'number_of_followers', 'following'])
    print("Finished scraping tweets")


    columns = ['user', 'tweets', 'time', 'favorites', 'retweets', 'number_of_followers', 'following']

    tweets_df_last100hours = pd.read_csv('assets/runs/Last100hoursUpdate.csv')
    tweets_df_last100hours = tweets_df_last100hours[columns]
    tweets_df_last1hours = tweets_df_last1hours[columns]
    df170 = pd.concat([tweets_df_last100hours, tweets_df_last1hours], ignore_index=True)
    df170 = df170.drop_duplicates()
    df170.to_csv('assets/runs/Last100hoursUpdate.csv')

    apikey = 'Tei1cl6Qncsr03oPvcFN0eW5PfaYZGIf7ZtoX6TBKB7G8UvJTzQsYhB92vzi9NdN'
    secret = 'kePezzaWGED3MYgzWYldFCfgNg69Tcm7vv1IEvFazKTKrT54fOyyTONualYiHnBF'


    #Authenticating

    client = Client(apikey, secret)# Get the date 1 week ago from today
    one_week_ago = datetime.now() - timedelta(weeks=10)
    one_week_ago_str = one_week_ago.strftime("%d %b %Y")

    #getting daily data
    historical = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1HOUR, one_week_ago_str)
    Original_df_historical = pd.DataFrame(historical)
    df_historical= Original_df_historical.copy()

    df_historical.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

    #changing time to standard units
    df_historical['Open Time'] = pd.to_datetime(df_historical['Open Time']/1000, unit='s')
    df_historical['Close Time'] = pd.to_datetime(df_historical['Close Time']/1000, unit='s')
    df_historical['Close Time'] = df_historical['Close Time'].values.astype('datetime64[h]')

    #changing numbers to integer
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    df_historical[numeric_columns] = df_historical[numeric_columns].apply(pd.to_numeric, axis=1)
    df_historical2_hourly =df_historical.set_index('Close Time')

    #calculating Moving average
    # Example 2: 
    numbers = df_historical2_hourly['Close']
    window_size = 3

    i = 0
    moving_averages = []
    for i in range(len(numbers)):
        try:
    #         this_window = numbers[i : i + window_size]
            this_window = numbers[i - window_size: i]
    #Get current window
            window_average = sum(this_window) / window_size
            moving_averages.append(window_average)
            i += 1
        except ValueError:
            print(NaN)

    #Adding the moving averages column
    df_historical2_hourly['3MovingAverage'] = moving_averages


    #calculating Moving average
    # Example 2: 
    numbers = df_historical2_hourly['Close']
    window_size = 5

    i = 0
    moving_averages = []
    for i in range(len(numbers)):
        try:
    #         this_window = numbers[i : i + window_size]
            this_window = numbers[i - window_size: i]
    #Get current window
            window_average = sum(this_window) / window_size
            moving_averages.append(window_average)
            i += 1
        except ValueError:
            print(NaN)
    #print(moving_averages)

    #Adding the moving averages column
    df_historical2_hourly['5MovingAverage'] = moving_averages


    #adding function to calcuate sentiment as postive, negative, or neutral. This is later used to calculate our bullratio score
    def getSentiment(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'


    df170['followers_following_ratio'] = df170['number_of_followers'] / df170['following']
    df170['2x_retweets_+_favorites'] = 2 * df170['retweets'] + df170['favorites']
    df170['polarity'] = df170['tweets'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment.polarity))
    df170['sentiment'] = df170['polarity'].apply(getSentiment)
    np.seterr(divide = 'ignore') 
    df170['W1 Score'] = (np.log(df170['followers_following_ratio'])+1)* np.log(df170['2x_retweets_+_favorites'])*(df170['polarity'])
    df170 = df170.fillna(0)
    df170.replace([np.inf, -np.inf, -0], 0, inplace=True)

    df170['time'] = pd.to_datetime(df170['time'], utc=True)

    df170['time'] = pd.to_datetime(df170['time'])
    df170['hour'] = df170['time'].dt.hour
    df170['day'] = pd.to_datetime(df170['time'].dt.date)
    df170['time'] = df170['day'] + pd.to_timedelta(df170['hour'], unit='h')


    Bullratio_df = df170.groupby('time').sum()



    bull_ratio_list = []
    Bullratio_df = Bullratio_df.reset_index()

    for day in Bullratio_df['time']:
      #print(day)
        filtered_df = df170.loc[(df170['time'] == day)]
        try:
            bull_ratio_list.append(filtered_df['sentiment'].value_counts()[1]/filtered_df['sentiment'].value_counts()[2]) #KEY NEED TO CHANGE TO [1] and [2] later
        except:
            bull_ratio_list.append(filtered_df['sentiment'].value_counts()[0]/filtered_df['sentiment'].value_counts()[0]) #KEY NEED TO CHANGE TO [1] and [2] later


    Bullratio_df['Bull_Ratio'] = bull_ratio_list
    df170['Bull_ratio'] = df170['time'].map(Bullratio_df.reset_index().set_index('time')['Bull_Ratio'])
    df170['W Score With Bull Ratio'] = df170['W1 Score'] * df170['Bull_ratio']

    Df_hourly = df170.groupby('time').mean()

    Df_hourly_merge = pd.merge(Df_hourly, df_historical2_hourly, how ='left', left_index=True, right_index=True, indicator = True) 

    #calculating Moving average
    # Example 2: 
    numbers = Df_hourly_merge['W Score With Bull Ratio']
    n = 3
    x = 0
    JMJ_3Hmoving_averages = []

    for i in range(len(numbers)):
        try:
            percentage = ((Df_hourly_merge['W Score With Bull Ratio'][x]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1])))*Df_hourly_merge['Close'][x] + ((Df_hourly_merge['W Score With Bull Ratio'][x-1]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1])))*Df_hourly_merge['Close'][x-1]+ ((Df_hourly_merge['W Score With Bull Ratio'][x-2]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1]))) *Df_hourly_merge['Close'][x-2]     
            JMJ_3Hmoving_averages.append(percentage)
            x = x + 1

        except ValueError:
            print(NaN)

    #calculating Moving average
    # Example 2: 
    numbers = Df_hourly_merge['W Score With Bull Ratio']
    n = 5
    x = 0
    JMJ_5Hmoving_averages = []

    for i in range(len(numbers)):
      try:
          percentage = ((Df_hourly_merge['W Score With Bull Ratio'][x]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1])))*Df_hourly_merge['Close'][x] + ((Df_hourly_merge['W Score With Bull Ratio'][x-1]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1])))*Df_hourly_merge['Close'][x-1] + ((Df_hourly_merge['W Score With Bull Ratio'][x-2]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1]))) *Df_hourly_merge['Close'][x-2] + ((Df_hourly_merge['W Score With Bull Ratio'][x-3]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1]))) *Df_hourly_merge['Close'][x-3] + ((Df_hourly_merge['W Score With Bull Ratio'][x-4]) / (sum(Df_hourly_merge['W Score With Bull Ratio'][x-n+1:x+1]))) *Df_hourly_merge['Close'][x-4]
          JMJ_5Hmoving_averages.append(percentage)
          x = x + 1

      except ValueError:
          print(NaN)


    Df_hourly_merge['JMJ_3HMoving_averages'] = JMJ_3Hmoving_averages
    Df_hourly_merge['JMJ_5HMoving_averages'] = JMJ_5Hmoving_averages
    Df_hourly_merge.replace([np.inf, -np.inf, -0], 0, inplace=True)
    # Df_hourly_merge.drop('Unnamed: 0', inplace=True, axis=1) #KEY may or may not need it

    Df_hourly_merge['Signal'] = 0.0
    Df_hourly_merge['Signal'] = np.where(Df_hourly_merge['3MovingAverage'] >Df_hourly_merge['5MovingAverage'], 1.0, 0.0)
    Df_hourly_merge['Position'] = Df_hourly_merge['Signal'].diff()


    Df_hourly_merge['Signal35JMJ'] = 0.0
    Df_hourly_merge['Signal35JMJ'] = np.where(Df_hourly_merge['JMJ_3HMoving_averages'] >Df_hourly_merge['JMJ_5HMoving_averages'], 1.0, 0.0)
    Df_hourly_merge['Position35JMJ'] = Df_hourly_merge['Signal35JMJ'].diff()


    prediction = (Df_hourly_merge.shift(-1)['Close'] >= Df_hourly_merge['Close'])
    prediction = prediction.iloc[:-1]
    Df_hourly_merge['Actual_Label'] = prediction.astype(int)
    Df_hourly_merge['Actual_Label'][-1]=3
    Df_hourly_merge.to_csv('assets/Final_GO_Live_PCHourly20192022_ActualLabel.csv')                                     

    # Calculate the start and end dates for the last day
    end_date = datetime.now().date() + timedelta(days=1)
    start_date = end_date - timedelta(days=5)

    # Convert dates to string format
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')                                     


    endpoint_url = "https://paper-api.alpaca.markets"
    api = REST("PKN12PE5ALWR43EC685W", "qdu7NgYDh6gV2iGaFPIgoboplKKTjv98qlITxW8C", endpoint_url)

    crypto_news = api.get_news(["BTCUSD","BTC","$BTC","ETHUSD","ETH","$ETH","DOGEUSD","DOGE","$DOGE"], start_date_str, end_date_str, limit=41000)
    mkt_news = api.get_news(["SPY","NASDAQ","VIX^",], start_date_str, end_date_str, limit=41000)

    crypto_news_list = []

    for story in crypto_news:
        crypto_news_dict = {}
        crypto_news_dict["time"] = story.created_at
        crypto_news_dict["Crypto Headline"] = story.headline
        crypto_news_list.append(crypto_news_dict)

    crypto_news_df = pd.DataFrame(crypto_news_list)

    mkt_news_list = []

    for story in mkt_news:
        mkt_news_dict = {}
        mkt_news_dict["time"] = story.created_at
        mkt_news_dict["Mkt Headline"] = story.headline
        mkt_news_list.append(mkt_news_dict)

    mkt_news_df = pd.DataFrame(mkt_news_list)

    # Calculate sentiment scores for each headline
    crypto_sentiments = []

    for headline in crypto_news_df['Crypto Headline']:
        blob = TextBlob(str(headline))
        crypto_sentiment = blob.sentiment.polarity
        crypto_sentiments.append(crypto_sentiment)

    crypto_news_df['Crypto Sentiment'] = crypto_sentiments

    mkt_sentiments = []

    for headline in mkt_news_df['Mkt Headline']:
        blob = TextBlob(str(headline))
        mkt_sentiment = blob.sentiment.polarity
        mkt_sentiments.append(mkt_sentiment)

    mkt_news_df['Mkt Sentiment'] = mkt_sentiments

    crypto_sentiment_df = crypto_news_df.resample('H', on='time').mean()[['Crypto Sentiment']].fillna(0)
    mkt_sentiment_df = mkt_news_df.resample('H', on='time').mean()[['Mkt Sentiment']].fillna(0)

    # Merge Dataframes & Filter
    market_sentiment_df = pd.merge(mkt_sentiment_df, crypto_sentiment_df, how='outer', on='time').fillna(0)
    market_sentiment_train_df = market_sentiment_df

    #Good Until here
    #days to hours

    SMA_st=11
    SMA_lt=21
    WMA_st=25
    WMA_lt=49
    EMA_st=29
    EMA_lt=57

    time_period_1 =SMA_st*24
    time_period_2 =SMA_lt*24
    time_period_1a =WMA_st*24
    time_period_2a =WMA_lt*24
    time_period_1b =EMA_st*24
    time_period_2b =EMA_lt*24


    # BTC_Ticker = yf.Ticker("BTC-USD")
    # result_df_sma = BTC_Ticker.history(period="max")   
    result_df_sma = df_historical2_hourly.copy()                                  
    result_df_sma["sma1"] = result_df_sma["Close"].rolling(window=time_period_1).mean()
    result_df_sma["sma2"] = result_df_sma["Close"].rolling(window=time_period_2).mean()                                     

    weights_1 = np.arange(1, time_period_1a + 1)
    weights_2 = np.arange(1, time_period_2a + 1)

    result_df_sma["wma1"] = result_df_sma["Close"].rolling(window=time_period_1a).apply(lambda x: np.sum(x * weights_1) / np.sum(weights_1), raw=True)
    result_df_sma["wma2"] = result_df_sma["Close"].rolling(window=time_period_2a).apply(lambda x: np.sum(x * weights_2) / np.sum(weights_2), raw=True)


    result_df_sma["ema1"] = result_df_sma["Close"].ewm(span=time_period_1b, adjust=False).mean()
    result_df_sma["ema2"] = result_df_sma["Close"].ewm(span=time_period_2b, adjust=False).mean()

    # result_df_sma = result_df_sma.drop(['Dividends', 'Stock Splits'], axis=1)

    Final_Dataframe = result_df_sma.rename(columns={
                                          "sma1": "Historically Optimal SMA(s-t)", "sma2": "Historically Optimal SMA(l-t)", 
                                         "wma1": "Historically Optimal WMA(s-t)", "wma2": "Historically Optimal WMA(l-t)", 
                                          "ema1": "Historically Optimal EMA(s-t)", "ema2": "Historically Optimal EMA(l-t)" 
                                         })

    Final_Dataframe

    # # resample the data to hourly frequency and interpolate missing values
    # Final_Dataframe_hourly = Final_Dataframe.resample('H').interpolate()

    # # calculate the hourly moving averages using a rolling window
    # Final_Dataframe_hourly_ma = Final_Dataframe_hourly.rolling(window=24, min_periods=1).mean()

    # # remove the first 23 rows as they only have partial data for the first day
    # Final_Dataframe_hourly_ma = Final_Dataframe_hourly_ma.iloc[23:]

    df_1 = Final_Dataframe.reset_index()
    df_1['Open Time'] = pd.to_datetime(df_1['Open Time']).dt.tz_localize('UTC')

    filename = "assets/Final_GO_Live_PCHourly20192022_ActualLabel.csv"
    df_2 = pd.read_csv(filename)
    df_2 = df_2.reset_index()
    df_2['time'] = pd.to_datetime(df_2['time']).dt.tz_localize('UTC')


    #Merge the two dataframes
    merged_df = pd.merge(df_1, df_2, left_on='Open Time', right_on='time')
    merged_df = merged_df.drop(['Close Time','Open Time_y', 'Open_y', 'High_y', 'Low_y', 'Close_y', 'Volume_y',
           'Quote Asset Volume_y', 'Number of Trades_y', 'TB Base Volume_y',
           'TB Quote Volume_y', 'Ignore_y','Ignore_x', '3MovingAverage_y', '5MovingAverage_y'], axis=1)

    merged_train_df = merged_df.rename(columns={'Open Time_x': 'Datetime','Open_x': 'Open', 'High_x': 'High', 'Low_x': 'Low', 
                 'Close_x': 'Close', 'Volume_x': 'Volume','Quote Asset Volume_x':'Quote Asset Volume', 'Signal': 'SMA Signal 35', 
                 'Signal35JMJ': 'JMJ Signal 35', 'Number of Trades_x': '# of Hourly Trades', 'TB Base Volume_x':'TB Base Volume',
                  'TB Quote Volume_x':'TB Quote Volume',                             
                 'JMJ_3HMoving_averages': 'JMJ 3 Hour MA', 'JMJ_5HMoving_averages': 'JMJ 5 Hour MA', 
                 'favorites': 'Twitter Hourly Favorites', 'retweets': 'Twitter Hourly Retweets', 
                 'number_of_followers': 'Twitter Hourly Follower Exposure', 'following': 'Twitter Hourly Following Exposure',
                'followers_following_ratio': 'Twitter Hourly Follower to Following Ratio', '2x_retweets_+_favorites': 
                 'Twitter Hourly 2x Retweets + Favorites', 'polarity': 'Twitter Hourly Polarity Score', 'W1 Score': 'Twitter W1 Score',
                'Bull_ratio': 'Twitter Hourly Bull Ratio', 'W Score With Bull Ratio': 'Twitter Hourly Weighted Bull Ratio', '3MovingAverage_x':
                '3 Hour MA', '5MovingAverage_x': '5 Hour MA', 'Actual_Label': "Actual Label", 'Position': 'SMA Position 35', 'Position35JMJ': 'JMJ Position 35'})
    merged_train_df = pd.merge(merged_train_df, market_sentiment_train_df, how='left', left_on='Datetime', right_on='time')

    last_valid_index_mkt = merged_train_df['Mkt Sentiment'].last_valid_index()
    last_valid_index_crypto = merged_train_df['Crypto Sentiment'].last_valid_index()

    last_valid_index = max(last_valid_index_mkt, last_valid_index_crypto)

    merged_train_df.loc[last_valid_index + 1:, ['Mkt Sentiment', 'Crypto Sentiment']] = merged_train_df.loc[last_valid_index + 1:, ['Mkt Sentiment', 'Crypto Sentiment']].fillna(0)



    def calculate_moving_averages(df, columns, SMA_st, SMA_lt, WMA_st, WMA_lt, EMA_st, EMA_lt):
        df_copy = df.copy() 
        for column in columns:
            # Calculate moving average
            df_copy[column+' SMA(s-t)'] = df_copy[column].rolling(window=(SMA_st)).mean()
            df_copy[column+' SMA(l-t)'] = df_copy[column].rolling(window=(SMA_lt)).mean()

            # Calculate weighted moving average
            weights = np.arange(1, WMA_st+1)
            df_copy[column+' WMA(s-t)'] = df_copy[column].rolling(window=(WMA_st)).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
            weights = np.arange(1, WMA_lt+1)
            df_copy[column+' WMA(l-t)'] = df_copy[column].rolling(window=(WMA_lt)).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)

            # Calculate exponential moving average
            df_copy[column+' EMA(s-t)'] = df_copy[column].ewm(span=(EMA_st), adjust=False).mean()
            df_copy[column+' EMA(l-t)'] = df_copy[column].ewm(span=(EMA_lt), adjust=False).mean()

        return df_copy    

    columns = ['Twitter Hourly Favorites',
           'Twitter Hourly Retweets', 'Twitter Hourly Follower Exposure',
           'Twitter Hourly Following Exposure',
           'Twitter Hourly Follower to Following Ratio',
           'Twitter Hourly 2x Retweets + Favorites',
           'Twitter Hourly Polarity Score', 'Twitter W1 Score',
           'Twitter Hourly Bull Ratio', 'Twitter Hourly Weighted Bull Ratio',
           'Quote Asset Volume', '# of Hourly Trades', 'TB Base Volume',
           'TB Quote Volume']

    MA_train_DF = calculate_moving_averages(merged_train_df,columns, SMA_st, SMA_lt, WMA_st, WMA_lt, EMA_st, EMA_lt)

    MA_train_DF.to_csv('assets/Final_Go_Live_Jesus_MA_train_dataframe.csv')


    requests_args = {'headers': {
        'authority': 'trends.google.com',
        'method': 'GET',
        'path': '/trends/api/autocomplete/BTC%20Bitcoin?hl=en-US&tz=420',
        'scheme': 'https',
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,ko;q=0.8',
        'cookie':'__utmc=10102256; __utmz=10102256.1679329027.1.1.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utma=10102256.1496328958.1679329018.1679329027.1679329027.2; __utmt=1; __utmb=10102256.11.9.1679336351095; S=billing-ui-v3=pYYo7r79iXi-TZTh2uUj15DOGcTJrAws:billing-ui-v3-efe=pYYo7r79iXi-TZTh2uUj15DOGcTJrAws; __Secure-ENID=6.SE=flR0TrAbRYorkBrH_5FaKbGaC4eZvnQ0YV4cD9BZHC6M9N_ivtbL2cppn-2MZz9HutuIp_vUPDCzZ63XqJKAUHM49e7IZpaRRB1VKJRTB7w3U2poH9-LzgaFOtQeN9rlHHqgIDwHKHw7lW_Bw9ZOG2dqkdlid24uevth0tD23m_x42O1QOa5ffAHfDkViUGyrfKnyvyH2_hvtsTpyXoqvl2riS3K0GdsFVFmy113H-fWqJ8y4FNuqek16Bfw-ZHDk8Ni-Nln; ab.storage.deviceId.240e177d-4779-41c2-b484-3af37ffa8685=%7B%22g%22%3A%223f51bc2b-caa1-0f11-c497-746247a5f893%22%2C%22c%22%3A1678796013500%2C%22l%22%3A1678796013500%7D; SID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHebW0faNP6Eukz0zkGHvp_QA.; __Secure-1PSID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHeqtrfQSOugwhU9s5bk3dzyg.; __Secure-3PSID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHef930e9OFbmwpbP-Yv2fKtA.; HSID=AMurgpyEkMg3Ewx9m; SSID=AhmJKGdjfaEjiJ5nZ; APISID=fDLYhRj3xasSGf5q/Ad1BdfZ2k2fOmltI8; SAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; __Secure-1PAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; __Secure-3PAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; AEC=ARSKqsLvV86pgpia_Fui1LWKv-UWOM8xYLpt_dgafcs0X2yvdMnmWqujTg; SEARCH_SAMESITE=CgQI7JcB; _gid=GA1.3.575088293.1679329020; OTZ=6950417_84_88_104280_84_446940; NID=511=Vx7_pxhLT04OEwBWEBi5lQv2D4DuN2dVetQAteO_n0rfBn5w_8DSUp52HJhUUYm25ElyMVZAfmBvb-46sSRcdpJMpfRQOMRP___c4ZUn2fNSnbzvL1dezF4IPWd6G0WBOU-MwNxfGEHEfvzrLWAnZmnguulJd_5GNA2FMUxpHp8v5J74mMB2vECbvjh4xqBTx2ZbGb3DpTCurQuGijF-d08TIXyIyXr2NLBQgijSfJudwUiD1eUvPKYvyXWid7dL8XAzrl9ZihhYay5V4fYu_4pl4ZJgYFD8wdaZh0GUMfL1ByJh5uZjSlcktN-DzFNmaAzilTeE-qsWC3_AYXX_JCl0qtp1F8kzV4dFzzWryK_1cEnb6ehjFdO_RN6ac7McPsiPv6XJrFj9uhfEew; 1P_JAR=2023-3-20-18; _ga=GA1.3.1496328958.1679329018; _gat_gtag_UA_4401283=1; _ga_VWZPXDNJJB=GS1.1.1679335973.2.1.1679336393.0.0.0; SIDCC=AFvIBn9qaKwR_XVYdzqts09MTAMaYTUaTMvtbcG0EiOQH5JnqgLD-BnE1RLYCJIZYdavxbzROw; __Secure-1PSIDCC=AFvIBn8XCUaH5nBAAB8Gw5VOOfZDFeOHkiXzImYfcFEACQ_I3S1nAcAiHh_DEukR9UcF5SjwSNs; __Secure-3PSIDCC=AFvIBn_Iq0jwOb1MB3JgagxDjxWkw2wM4Q3-uE4Inahw31SC7jIXVWR5BqIT65rgnCeqguOIEPmf',
        'referer': 'https://trends.google.com/trends/explore?date=2019-01-01%202022-10-01&q=BTC%20Bitcoin&hl=en',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-arch': '"arm"',
        'sec-ch-ua-bitness': '"64"',
        'sec-ch-ua-full-version': '"111.0.5563.64"',
        'sec-ch-ua-full-version-list': '"Google Chrome";v="111.0.5563.64", "Not(A:Brand";v="8.0.0.0", "Chromium";v="111.0.5563.64"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '',
        'sec-ch-ua-platform': '"macOS"',
        'sec-ch-ua-platform-version': '"12.0.1"',
        'sec-ch-ua-wow64': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'x-client-data': 'CLSVywE='}
    }

    """Specify start and end date as well es the required keyword for your query"""
    # end_date = datetime.now().date()
    end_date = datetime.now().date() + timedelta(days=1)
    start_date = end_date - timedelta(days=20)
    keyword_list = ["Bitcoin", "BTC"] # If you add a second string, minor adjustments in the code have to be made

    """Since we want weekly data for our query, we will create lists which include 
    the weekly start and end date in the specified timeframe - 2018.01.01 to 2019.5.01"""

    weekly_date_list = []

    #Adds the start date as first entry in our weekly_date_list
    start_date_temp = start_date
    weekly_date_list.append(start_date_temp)

    #This will return in list of weekly datetime.date objects - except the end date
    while start_date_temp + timedelta(days=7) <= end_date:
        start_date_temp += timedelta(days=7)
        weekly_date_list.append(start_date_temp)

    #This will add the end date to the weekly_date list. We now have a complete list in the specified timeframe
    if start_date_temp + timedelta(days=7) > end_date:
        weekly_date_list.append(end_date)


    """Now we can start to downloading the data via Google Trends API
    therefore we have to specify a key which includes the start date
    and the end-date with T00 as string for hourly data request"""

    #This List will contain pandas Dataframes of weekly data with the features "date",
    #"keyword"(which contains weekly scaling bettwen 0 and 100), "isPartial".
    #Up to this point, the scaling is not correct.
    interest_list = []

    #Here we download the data and print the current status of the process
    for i in range(len(weekly_date_list)-1):
        requests_args = {'headers': {
        'authority': 'trends.google.com',
        'method': 'GET',
        'path': '/trends/api/autocomplete/BTC%20Bitcoin?hl=en-US&tz=420',
        'scheme': 'https',
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,ko;q=0.8',
        'cookie':'__utmc=10102256; __utmz=10102256.1679329027.1.1.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utma=10102256.1496328958.1679329018.1679329027.1679329027.2; __utmt=1; __utmb=10102256.11.9.1679336351095; S=billing-ui-v3=pYYo7r79iXi-TZTh2uUj15DOGcTJrAws:billing-ui-v3-efe=pYYo7r79iXi-TZTh2uUj15DOGcTJrAws; __Secure-ENID=6.SE=flR0TrAbRYorkBrH_5FaKbGaC4eZvnQ0YV4cD9BZHC6M9N_ivtbL2cppn-2MZz9HutuIp_vUPDCzZ63XqJKAUHM49e7IZpaRRB1VKJRTB7w3U2poH9-LzgaFOtQeN9rlHHqgIDwHKHw7lW_Bw9ZOG2dqkdlid24uevth0tD23m_x42O1QOa5ffAHfDkViUGyrfKnyvyH2_hvtsTpyXoqvl2riS3K0GdsFVFmy113H-fWqJ8y4FNuqek16Bfw-ZHDk8Ni-Nln; ab.storage.deviceId.240e177d-4779-41c2-b484-3af37ffa8685=%7B%22g%22%3A%223f51bc2b-caa1-0f11-c497-746247a5f893%22%2C%22c%22%3A1678796013500%2C%22l%22%3A1678796013500%7D; SID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHebW0faNP6Eukz0zkGHvp_QA.; __Secure-1PSID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHeqtrfQSOugwhU9s5bk3dzyg.; __Secure-3PSID=UQibUU0kksn5OWD6mOj4eVEi0VAphmoHoZvBUeJPUUBeTOHef930e9OFbmwpbP-Yv2fKtA.; HSID=AMurgpyEkMg3Ewx9m; SSID=AhmJKGdjfaEjiJ5nZ; APISID=fDLYhRj3xasSGf5q/Ad1BdfZ2k2fOmltI8; SAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; __Secure-1PAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; __Secure-3PAPISID=Y6UjmWzruWzZmgT7/AE9ynEOWFAq-gJSTA; AEC=ARSKqsLvV86pgpia_Fui1LWKv-UWOM8xYLpt_dgafcs0X2yvdMnmWqujTg; SEARCH_SAMESITE=CgQI7JcB; _gid=GA1.3.575088293.1679329020; OTZ=6950417_84_88_104280_84_446940; NID=511=Vx7_pxhLT04OEwBWEBi5lQv2D4DuN2dVetQAteO_n0rfBn5w_8DSUp52HJhUUYm25ElyMVZAfmBvb-46sSRcdpJMpfRQOMRP___c4ZUn2fNSnbzvL1dezF4IPWd6G0WBOU-MwNxfGEHEfvzrLWAnZmnguulJd_5GNA2FMUxpHp8v5J74mMB2vECbvjh4xqBTx2ZbGb3DpTCurQuGijF-d08TIXyIyXr2NLBQgijSfJudwUiD1eUvPKYvyXWid7dL8XAzrl9ZihhYay5V4fYu_4pl4ZJgYFD8wdaZh0GUMfL1ByJh5uZjSlcktN-DzFNmaAzilTeE-qsWC3_AYXX_JCl0qtp1F8kzV4dFzzWryK_1cEnb6ehjFdO_RN6ac7McPsiPv6XJrFj9uhfEew; 1P_JAR=2023-3-20-18; _ga=GA1.3.1496328958.1679329018; _gat_gtag_UA_4401283=1; _ga_VWZPXDNJJB=GS1.1.1679335973.2.1.1679336393.0.0.0; SIDCC=AFvIBn9qaKwR_XVYdzqts09MTAMaYTUaTMvtbcG0EiOQH5JnqgLD-BnE1RLYCJIZYdavxbzROw; __Secure-1PSIDCC=AFvIBn8XCUaH5nBAAB8Gw5VOOfZDFeOHkiXzImYfcFEACQ_I3S1nAcAiHh_DEukR9UcF5SjwSNs; __Secure-3PSIDCC=AFvIBn_Iq0jwOb1MB3JgagxDjxWkw2wM4Q3-uE4Inahw31SC7jIXVWR5BqIT65rgnCeqguOIEPmf',
        'referer': 'https://trends.google.com/trends/explore?date=2019-01-01%202022-10-01&q=BTC%20Bitcoin&hl=en',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-arch': '"arm"',
        'sec-ch-ua-bitness': '"64"',
        'sec-ch-ua-full-version': '"111.0.5563.64"',
        'sec-ch-ua-full-version-list': '"Google Chrome";v="111.0.5563.64", "Not(A:Brand";v="8.0.0.0", "Chromium";v="111.0.5563.64"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '',
        'sec-ch-ua-platform': '"macOS"',
        'sec-ch-ua-platform-version': '"12.0.1"',
        'sec-ch-ua-wow64': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'x-client-data': 'CLSVywE='}
        }

        key =str(weekly_date_list[i])+"T00 "+str(weekly_date_list[i+1])+"T00"
        p = TrendReq(requests_args=requests_args)
        p.build_payload(kw_list=keyword_list, timeframe=key)
        interest=p.interest_over_time()
        interest_list.append(interest)
        print("GoogleTrends Call {} of {} : Timeframe: {} "\
              .format(i+1, len(weekly_date_list)-1, key )
              )   

    df = pd.concat(interest_list)
    df.drop(labels = "isPartial", axis = 1, inplace = True)
    df = df.reset_index()
    df.rename(columns={"date": "Datetime", "Bitcoin": "Bitcoin_Google_Trend_Score", "BTC": "BTC_Google_Trend_Score"}, inplace=True)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)

    df_train = df


    news_train_df = pd.read_csv('assets/Final_Go_Live_Jesus_MA_train_dataframe.csv')
    news_train_df['Datetime'] = news_train_df['Datetime'].str[:-6]
    news_train_df['Datetime'] = pd.to_datetime(news_train_df['Datetime'])
    # news_train_df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # news_train_df['Datetime'].dt.tz_localize(None)
    news_train_df.set_index('Datetime', inplace=True)


    df_train_merge = pd.merge(df_train, news_train_df, how ='left', left_index=True, right_index=True)

    df_train_merge.to_csv('assets/Final_Go_Live_Jesus_google_trends_train_jan_01_2019_to_nov_30_2021.csv')


    # Extracting milestone 2 dataframe
    Df_hourly_merge = pd.read_csv('assets/Final_GO_Live_PCHourly20192022_ActualLabel.csv')

    #Ensuring we have dat and time correct
    Df_hourly_merge['time'] = pd.to_datetime(Df_hourly_merge['time'])
    Df_hourly_merge =Df_hourly_merge.reset_index().set_index('time')

    Df_hourly_merge = Df_hourly_merge[['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'hour', 'Bull_ratio', 'W Score With Bull Ratio',
           'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
           'Quote Asset Volume', 'Number of Trades', 'TB Base Volume',
           'TB Quote Volume', 'Ignore', '3MovingAverage', '5MovingAverage',
           'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 'Signal',
           'Position', 'Signal35JMJ', 'Position35JMJ', 'Actual_Label']]

    Df_JT = pd.read_csv('assets/Final_Go_Live_Jesus_google_trends_train_jan_01_2019_to_nov_30_2021.csv')
    #Convert date into hourly
    Df_JT['time'] = pd.to_datetime(Df_JT['Datetime'])
    Df_JT['hour'] = Df_JT['time'].dt.hour
    Df_JT['day'] = pd.to_datetime(Df_JT['time'].dt.date)
    Df_JT['time'] = Df_JT['day'] + pd.to_timedelta(Df_JT['hour'], unit='h')
    Df_JT =Df_JT.reset_index().set_index('time')

    Df_JT = Df_JT[['Bitcoin_Google_Trend_Score',
           'BTC_Google_Trend_Score','Mkt Sentiment',
           'Crypto Sentiment','Historically Optimal SMA(s-t)',
           'Historically Optimal SMA(l-t)', 
           'Historically Optimal WMA(s-t)', 'Historically Optimal WMA(l-t)',
            'Historically Optimal EMA(s-t)',
           'Historically Optimal EMA(l-t)',  'Twitter Hourly Favorites SMA(s-t)',
           'Twitter Hourly Favorites SMA(l-t)',
           'Twitter Hourly Favorites WMA(s-t)',
           'Twitter Hourly Favorites WMA(l-t)',
           'Twitter Hourly Favorites EMA(s-t)',
           'Twitter Hourly Favorites EMA(l-t)',
           'Twitter Hourly Retweets SMA(s-t)',
           'Twitter Hourly Retweets SMA(l-t)',
           'Twitter Hourly Retweets WMA(s-t)',
           'Twitter Hourly Retweets WMA(l-t)',
           'Twitter Hourly Retweets EMA(s-t)',
           'Twitter Hourly Retweets EMA(l-t)',
           'Twitter Hourly Follower Exposure SMA(s-t)',
           'Twitter Hourly Follower Exposure SMA(l-t)',
           'Twitter Hourly Follower Exposure WMA(s-t)',
           'Twitter Hourly Follower Exposure WMA(l-t)',
           'Twitter Hourly Follower Exposure EMA(s-t)',
           'Twitter Hourly Follower Exposure EMA(l-t)',
           'Twitter Hourly Following Exposure SMA(s-t)',
           'Twitter Hourly Following Exposure SMA(l-t)',
           'Twitter Hourly Following Exposure WMA(s-t)',
           'Twitter Hourly Following Exposure WMA(l-t)',
           'Twitter Hourly Following Exposure EMA(s-t)',
           'Twitter Hourly Following Exposure EMA(l-t)',
           'Twitter Hourly Follower to Following Ratio SMA(s-t)',
           'Twitter Hourly Follower to Following Ratio SMA(l-t)',
           'Twitter Hourly Follower to Following Ratio WMA(s-t)',
           'Twitter Hourly Follower to Following Ratio WMA(l-t)',
           'Twitter Hourly Follower to Following Ratio EMA(s-t)',
           'Twitter Hourly Follower to Following Ratio EMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites SMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites WMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites WMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites EMA(l-t)',
           'Twitter Hourly Polarity Score SMA(s-t)',
           'Twitter Hourly Polarity Score SMA(l-t)',
           'Twitter Hourly Polarity Score WMA(s-t)',
           'Twitter Hourly Polarity Score WMA(l-t)',
           'Twitter Hourly Polarity Score EMA(s-t)',
           'Twitter Hourly Polarity Score EMA(l-t)',
           'Twitter W1 Score SMA(s-t)', 'Twitter W1 Score SMA(l-t)',
           'Twitter W1 Score WMA(s-t)', 'Twitter W1 Score WMA(l-t)',
           'Twitter W1 Score EMA(s-t)', 'Twitter W1 Score EMA(l-t)',
           'Twitter Hourly Bull Ratio SMA(s-t)',
           'Twitter Hourly Bull Ratio SMA(l-t)',
           'Twitter Hourly Bull Ratio WMA(s-t)',
           'Twitter Hourly Bull Ratio WMA(l-t)',
           'Twitter Hourly Bull Ratio EMA(s-t)',
           'Twitter Hourly Bull Ratio EMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
           'Quote Asset Volume SMA(s-t)', 'Quote Asset Volume SMA(l-t)',
           'Quote Asset Volume WMA(s-t)', 'Quote Asset Volume WMA(l-t)',
           'Quote Asset Volume EMA(s-t)', 'Quote Asset Volume EMA(l-t)',
           '# of Hourly Trades SMA(s-t)', '# of Hourly Trades SMA(l-t)',
           '# of Hourly Trades WMA(s-t)', '# of Hourly Trades WMA(l-t)',
           '# of Hourly Trades EMA(s-t)', '# of Hourly Trades EMA(l-t)',
           'TB Base Volume SMA(s-t)', 'TB Base Volume SMA(l-t)',
           'TB Base Volume WMA(s-t)', 'TB Base Volume WMA(l-t)',
           'TB Base Volume EMA(s-t)', 'TB Base Volume EMA(l-t)',
           'TB Quote Volume SMA(s-t)', 'TB Quote Volume SMA(l-t)',
           'TB Quote Volume WMA(s-t)', 'TB Quote Volume WMA(l-t)',
           'TB Quote Volume EMA(s-t)', 'TB Quote Volume EMA(l-t)']]

    Df_hourly_mergewithJT = pd.merge(Df_hourly_merge, Df_JT, how ='left', left_index=True, right_index=True, indicator = True) 

    #Removing all Nana Columns to avoid errors later in Data modelling
    #Df_hourly_mergewithJT = Df_hourly_mergewithJT.dropna() #Key removedropna

    Df_hourly_mergewithJT.to_csv('assets/Final_Go_Live_Jesus_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv')




    ## Prediction:
    INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']


    testdata2022 ='assets/Final_Go_Live_Jesus_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv'


    files= [testdata2022]

    pkl_filename = 'model/Go_live_Stacking_LR_model_Final_80_20.pkl'
    with open(pkl_filename, 'rb') as file:
        Model2 = pickle.load(file)


    def _exponential_smooth(data, alpha):
        """
        Function that exponentially smooths dataset so values are less 'rigid'
        :param alpha: weight factor to weight recent values more
        """

        return data.ewm(alpha=alpha).mean()

    def _get_indicator_data(data):
        """
        Function that uses the finta API to calculate technical indicators used as the features
        :return:
        """

        for indicator in INDICATORS:
            ind_data = eval('TA.' + indicator + '(data)')
            if not isinstance(ind_data, pd.DataFrame):
                ind_data = ind_data.to_frame()
            data = data.merge(ind_data, left_index=True, right_index=True)
        data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

        # Also calculate moving averages for features
        data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
        data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
        data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
        data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

        # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
        data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

        return data

    UpperThreshold = 0.5
    LowerThresshold = 0.5

    for file,x in zip(files, ['testdata2022']):
        PredictDF= pd.read_csv(file)
    #     prediction = (PredictDF.shift(-1)['Close'] >= PredictDF['Close'])
    #     prediction = prediction.iloc[:-1]
    #     PredictDF['Actual_Label'] = prediction.astype(int)
    #     PredictDF= PredictDF.dropna()

        Indicatordata = _exponential_smooth(PredictDF[['Close', 'Open','High','Low','Volume']], 0.65)

        Indicatordatafinal = _get_indicator_data(Indicatordata)

        Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
        PredictDF = pd.merge(PredictDF, Indicatordatafinal, left_index=True, right_index=True)
        PredictDF = PredictDF.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)





        columns = ['favorites', 'retweets', 'number_of_followers', 'following',
               'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
               'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
               'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
               'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
               'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
               '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
               '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
               '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']


        def binary(value):
            if value > 0:
                return 1
            else:
                return 0

        for column in columns: 
            PredictDF['Binary{}'.format(column)]  = (PredictDF[column] - PredictDF[column].shift(1)).apply(binary)


        PredictDF = PredictDF.dropna()


        feature_names_JMJ = ['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open',
           'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume',
           'Number of Trades', 'TB Base Volume', 'TB Quote Volume',
           '3MovingAverage', '5MovingAverage', 'JMJ_3HMoving_averages',
            'Bitcoin_Google_Trend_Score', 'BTC_Google_Trend_Score',
           'JMJ_5HMoving_averages', 'Mkt Sentiment',
           'Crypto Sentiment', 'Historically Optimal SMA(s-t)',
           'Historically Optimal SMA(l-t)', 'Historically Optimal WMA(s-t)',
           'Historically Optimal WMA(l-t)', 'Historically Optimal EMA(s-t)',
           'Historically Optimal EMA(l-t)',
           'Twitter Hourly Favorites SMA(s-t)',
           'Twitter Hourly Favorites SMA(l-t)',
           'Twitter Hourly Favorites WMA(s-t)',
           'Twitter Hourly Favorites WMA(l-t)',
           'Twitter Hourly Favorites EMA(s-t)',
           'Twitter Hourly Favorites EMA(l-t)',
           'Twitter Hourly Retweets SMA(s-t)',
           'Twitter Hourly Retweets SMA(l-t)',
           'Twitter Hourly Retweets WMA(s-t)',
           'Twitter Hourly Retweets WMA(l-t)',
           'Twitter Hourly Retweets EMA(s-t)',
           'Twitter Hourly Retweets EMA(l-t)',
           'Twitter Hourly Follower Exposure SMA(s-t)',
           'Twitter Hourly Follower Exposure SMA(l-t)',
           'Twitter Hourly Follower Exposure WMA(s-t)',
           'Twitter Hourly Follower Exposure WMA(l-t)',
           'Twitter Hourly Follower Exposure EMA(s-t)',
           'Twitter Hourly Follower Exposure EMA(l-t)',
           'Twitter Hourly Following Exposure SMA(s-t)',
           'Twitter Hourly Following Exposure SMA(l-t)',
           'Twitter Hourly Following Exposure WMA(s-t)',
           'Twitter Hourly Following Exposure WMA(l-t)',
           'Twitter Hourly Following Exposure EMA(s-t)',
           'Twitter Hourly Following Exposure EMA(l-t)',
           'Twitter Hourly Follower to Following Ratio SMA(s-t)',
           'Twitter Hourly Follower to Following Ratio SMA(l-t)',
           'Twitter Hourly Follower to Following Ratio WMA(s-t)',
           'Twitter Hourly Follower to Following Ratio WMA(l-t)',
           'Twitter Hourly Follower to Following Ratio EMA(s-t)',
           'Twitter Hourly Follower to Following Ratio EMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites SMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites WMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites WMA(l-t)',
           'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
           'Twitter Hourly 2x Retweets + Favorites EMA(l-t)',
           'Twitter Hourly Polarity Score SMA(s-t)',
           'Twitter Hourly Polarity Score SMA(l-t)',
           'Twitter Hourly Polarity Score WMA(s-t)',
           'Twitter Hourly Polarity Score WMA(l-t)',
           'Twitter Hourly Polarity Score EMA(s-t)',
           'Twitter Hourly Polarity Score EMA(l-t)',
           'Twitter W1 Score SMA(s-t)', 'Twitter W1 Score SMA(l-t)',
           'Twitter W1 Score WMA(s-t)', 'Twitter W1 Score WMA(l-t)',
           'Twitter W1 Score EMA(s-t)', 'Twitter W1 Score EMA(l-t)',
           'Twitter Hourly Bull Ratio SMA(s-t)',
           'Twitter Hourly Bull Ratio SMA(l-t)',
           'Twitter Hourly Bull Ratio WMA(s-t)',
           'Twitter Hourly Bull Ratio WMA(l-t)',
           'Twitter Hourly Bull Ratio EMA(s-t)',
           'Twitter Hourly Bull Ratio EMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
           'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
           'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
           'Quote Asset Volume SMA(s-t)', 'Quote Asset Volume SMA(l-t)',
           'Quote Asset Volume WMA(s-t)', 'Quote Asset Volume WMA(l-t)',
           'Quote Asset Volume EMA(s-t)', 'Quote Asset Volume EMA(l-t)',
           '# of Hourly Trades SMA(s-t)', '# of Hourly Trades SMA(l-t)',
           '# of Hourly Trades WMA(s-t)', '# of Hourly Trades WMA(l-t)',
           '# of Hourly Trades EMA(s-t)', '# of Hourly Trades EMA(l-t)',
           'TB Base Volume SMA(s-t)', 'TB Base Volume SMA(l-t)',
           'TB Base Volume WMA(s-t)', 'TB Base Volume WMA(l-t)',
           'TB Base Volume EMA(s-t)', 'TB Base Volume EMA(l-t)',
           'TB Quote Volume SMA(s-t)', 'TB Quote Volume SMA(l-t)',
           'TB Quote Volume WMA(s-t)', 'TB Quote Volume WMA(l-t)',
           'TB Quote Volume EMA(s-t)', 'TB Quote Volume EMA(l-t)',
           '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
           '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV',
           '20 period CCI', '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21',
           'ema15', 'ema5', 'normVol', 'Binaryfavorites', 'Binaryretweets',
           'Binarynumber_of_followers', 'Binaryfollowing',
           'Binaryfollowers_following_ratio', 'Binary2x_retweets_+_favorites',
           'Binarypolarity', 'BinaryW1 Score', 'BinaryBull_ratio',
           'BinaryW Score With Bull Ratio', 'BinaryOpen', 'BinaryHigh',
           'BinaryLow', 'BinaryClose', 'BinaryVolume',
           'BinaryQuote Asset Volume', 'BinaryNumber of Trades',
           'BinaryTB Base Volume', 'BinaryTB Quote Volume',
           'Binary3MovingAverage', 'Binary5MovingAverage',
           'BinaryJMJ_3HMoving_averages', 'BinaryJMJ_5HMoving_averages',
           'Binary14 period RSI', 'BinaryMACD', 'BinarySIGNAL',
           'Binary14 period STOCH %K', 'BinaryMFV', 'Binary14 period ATR',
           'BinaryMOM', 'Binary14 period MFI', 'BinaryROC', 'BinaryOBV',
           'Binary20 period CCI', 'Binary14 period EMV', 'BinaryVIm',
           'BinaryVIp', 'Binaryema50', 'Binaryema21', 'Binaryema15',
           'Binaryema5']

        X_test = PredictDF[feature_names_JMJ]
        y_test = PredictDF['Actual_Label']

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_test)
        X_test_scaled = scaler.transform(X_test)

    #     predict_y1= np.argmax(stacking_classifier.predict_proba(X_test_scaled),axis=0)
        #predict_y=  Model2.predict_proba(X_test_scaled)[:,1]

        PredictDF['Prediction']= Model2.predict_proba(X_test_scaled)[:,1]


    dataframetime= pd.read_csv('assets/Final_Go_Live_Jesus_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv') 
    Final_prediction = pd.merge (dataframetime,PredictDF, left_index=True, right_index=True)
    Final_prediction.to_csv("results.csv")
    FinalPrediction2 = Final_prediction[['time','Prediction','Close_x']].iloc[-1]
    results_df = pd.DataFrame(columns=['time', 'Prediction', 'Close_x'])
    result_output_df = pd.DataFrame([FinalPrediction2])
    results_df = pd.concat([results_df, result_output_df], ignore_index=True)
    print(results_df)
    print(datetime.now())
    
    
# If you  want to start using the information to trade directly in Binance, you can uncoment the code below and link it to your Binance API

#     def signal(row):
#         if float(row['Prediction']) >= 0.525:
#             return 1
#         elif float(row['Prediction']) < 0.475:
#             return -1
#         else:
#             return 0

#     PredictionBinance = signal(results_df)

#     def max_buy_position():
#         to_use = float(exchange.fetch_balance().get('USDT').get('free'))
#         price = float(exchange.fetchTicker('BTC/USDT').get('last'))
#         decide_position_to_use = (to_use / price)
#         return decide_position_to_use, price

#     def max_sell_position():
#         to_use2 = float(exchange.fetch_balance().get('USDT').get('free'))
#         decide_position_to_use2 = to_use2
#         return decide_position_to_use2
#     in_position = False #meaning I already have the asset
#     bought_price = 27885.27 # will change, setup in beginning before running
#     sold_price = 0 #will change, setup in beginning before running
#     # Margin_Goal = 0.007 # 0.1%  to cover for binance fee


#     def execute_buy_sell_signals(df):
#         global in_position
#         global bought_price
#         global sold_price
#         print ('checking for buy and sell signals')


#         #BUY ORDER
#         if PredictionBinance = 1
#             print ('Recommendation to buy at probability of {}'.format(float(results_df['Prediction'])))
#             if not in_position:
#                 order = exchange.create_market_buy_order('BTC/USDT',max_buy_position()[0])
#                 print (order)
#                 print ('I have successfully bought')
#                 in_position = True
#                 bought_price = max_buy_position()[1]
#             else:
#                     print ('Already in position')

#          #SELL ORDER        
#         if PredictionBinance = -1
#             print ('Recommendation to sell at probability of {}'.format(float(results_df['Prediction'])))
#             if in_position:
#                 order = exchange.create_market_sell_order('BTC/USDT',max_sell_position())
#                 print (order)
#                 print ('I have successfully sold')
#                 in_position = False

#             else:
#                 print ("You aren't in positin, nothing to sell")    

#         elif PredictionBinance = 0:
#              print ('Recommendation to HOLD at probability {}'.format(float(results_df['Prediction'])))


def threaded_run_program():
    job_thread = threading.Thread(target=run_program)
    job_thread.start()

def schedule_one_minute_after_top_of_hour():
    now = datetime.now()
    next_hour = (now + timedelta(hours=1)).replace(minute=1, second=0, microsecond=0)
    initial_delay = (next_hour - now).total_seconds()
    
    # Run the program once after the initial delay
    time.sleep(initial_delay)
    threaded_run_program()
    
    # Schedule the repeated runs, starting one hour after the initial run
    schedule.every(1).hour.do(threaded_run_program)

schedule_one_minute_after_top_of_hour()

while True:
    schedule.run_pending()
    time.sleep(3600)  # Check every hour

Scraping tweets for the last 1 hour...
0
Finished scraping tweets



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


GoogleTrends Call 1 of 3 : Timeframe: 2023-03-16T00 2023-03-23T00 
GoogleTrends Call 2 of 3 : Timeframe: 2023-03-23T00 2023-03-30T00 
GoogleTrends Call 3 of 3 : Timeframe: 2023-03-30T00 2023-04-05T00 
                  time  Prediction   Close_x
0  2023-04-04 21:00:00    0.473491  28201.97
2023-04-04 23:49:14.172536
Wall time: 2min 32s
